In [84]:
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !jupyter nbextension enable --py --sys-prefix qgrid
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import qgrid
import g2p_client
from IPython.display import IFrame    
import re
from IPython.display import Javascript
from collections import Counter
from IPython.display import display, Markdown  



 


In [77]:

%env BIOONTOLOGY_API_KEY=fb76114e-2eff-4a05-9a2e-eba5b9fb6f0e

p115 = [{'description': '', 'start': 27100988L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'ARID1A', 'alt': 'T', 'ref': 'C', 'chromosome': '1', 'name': 'ARID1A '}, {'description': '', 'start': 204494718L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'MDM4', 'alt': 'G', 'ref': 'C', 'chromosome': '1', 'name': 'MDM4 '}, {'description': '', 'start': 30143341L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'ALK', 'alt': 'A', 'ref': 'G', 'chromosome': '2', 'name': 'ALK '}, {'description': '', 'start': 48028096L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'MSH6', 'alt': 'C', 'ref': 'G', 'chromosome': '2', 'name': 'MSH6 '}, {'description': '', 'start': 10084764L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'FANCD2', 'alt': 'G', 'ref': 'C', 'chromosome': '3', 'name': 'FANCD2 '}, {'description': '', 'start': 142185372L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'ATR', 'alt': 'G', 'ref': 'C', 'chromosome': '3', 'name': 'ATR '}, {'description': '', 'start': 178916923L, 'biomarker_type': 'indel', 'referenceName': 'GRCh37', 'geneSymbol': 'PIK3CA', 'alt': 'C', 'ref': 'CCAGTAG', 'chromosome': '3', 'name': 'PIK3CA '}, {'description': '', 'start': 117647566L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'ROS1', 'alt': 'C', 'ref': 'G', 'chromosome': '6', 'name': 'ROS1 '}, {'description': '', 'start': 87285753L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'NTRK2', 'alt': 'G', 'ref': 'C', 'chromosome': '9', 'name': 'NTRK2 '}, {'description': '', 'start': 108106483L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'ATM', 'alt': 'C', 'ref': 'G', 'chromosome': '11', 'name': 'ATM '}, {'description': '', 'start': 45618180L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'FANCM', 'alt': 'G', 'ref': 'C', 'chromosome': '14', 'name': 'FANCM '}, {'description': '', 'start': 45644454L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'FANCM', 'alt': 'C', 'ref': 'G', 'chromosome': '14', 'name': 'FANCM '}, {'description': '', 'start': 2110733L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'TSC2', 'alt': 'G', 'ref': 'C', 'chromosome': '16', 'name': 'TSC2 '}, {'description': 'GAIN Copy Number Ratio 3.0528', 'start': 39771328L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'IDO1', 'alt': '', 'ref': '', 'chromosome': '8', 'name': 'IDO1 '}, {'description': 'GAIN Copy Number Ratio 5.3306', 'start': 39792474L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'IDO2', 'alt': '', 'ref': '', 'chromosome': '8', 'name': 'IDO2 '}, {'description': 'LOSS Copy Number Ratio 0.3497', 'start': 21967751L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'CDKN2A', 'alt': '', 'ref': '', 'chromosome': '9', 'name': 'CDKN2A '}, {'description': 'GAIN Copy Number Ratio 7.5539', 'start': 38268656L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'FGFR1', 'alt': '', 'ref': '', 'chromosome': '8', 'name': 'FGFR1 '}, {'description': 'LOSS Copy Number Ratio 0.5662', 'start': 89623195L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'PTEN', 'alt': '', 'ref': '', 'chromosome': '10', 'name': 'PTEN '}, {'description': 'GAIN Copy Number Ratio 2.883', 'start': 69455873L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'CCND1', 'alt': '', 'ref': '', 'chromosome': '11', 'name': 'CCND1 '}, {'description': 'GAIN Copy Number Ratio 2.0415', 'start': 128748315L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'MYC', 'alt': '', 'ref': '', 'chromosome': '8', 'name': 'MYC '}]  # NOQA
p204 = [{'end': 41266125L, 'name': 'CTNNB1 p.T41I', 'start': 41266125L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CTNNB1', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'catenin (cadherin-associated protein), beta 1, 88kDa'}, {'end': 142188239L, 'name': 'ATR p.I2164M', 'start': 142188239L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ATR', 'alt': 'C', 'ref': 'T', 'chromosome': '3', 'description': 'ATR serine/threonine kinase'}, {'end': 89685310L, 'name': 'PTEN p.-70fs', 'start': 89685309L, 'biomarker_type': 'Frame_Shift_Ins', 'referenceName': 'GRCh37', 'geneSymbol': 'PTEN', 'alt': 'AATCT', 'ref': '-', 'chromosome': '10', 'description': 'phosphatase and tensin homolog'}, {'end': 41266125L, 'name': 'CTNNB1 p.T41I', 'start': 41266125L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CTNNB1', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'catenin (cadherin-associated protein), beta 1, 88kDa'}, {'end': 1584977L, 'name': 'P2RY8 p.A159T', 'start': 1584977L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'P2RY8', 'alt': 'T', 'ref': 'C', 'chromosome': 'X', 'description': 'purinergic receptor P2Y, G-protein coupled, 8'}, {'end': 170802031L, 'name': 'TNIK p.T1028A', 'start': 170802031L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'TNIK', 'alt': 'C', 'ref': 'T', 'chromosome': '3', 'description': 'TRAF2 and NCK interacting kinase'}, {'end': 118764345L, 'name': 'CXCR5 p.S31F', 'start': 118764345L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CXCR5', 'alt': 'T', 'ref': 'C', 'chromosome': '11', 'description': 'chemokine (C-X-C motif) receptor 5'}, {'end': 136071095L, 'name': 'ZRANB3 p.L310F', 'start': 136071095L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ZRANB3', 'alt': 'G', 'ref': 'C', 'chromosome': '2', 'description': 'zinc finger, RAN-binding domain containing 3'}, {'end': 228566043L, 'name': 'OBSCN p.V8839A', 'start': 228566043L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'OBSCN', 'alt': 'C', 'ref': 'T', 'chromosome': '1', 'description': 'obscurin, cytoskeletal calmodulin and titin-interacting RhoGEF'}, {'end': 2097401L, 'name': 'STK35 p.L328V', 'start': 2097401L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'STK35', 'alt': 'G', 'ref': 'C', 'chromosome': '20', 'description': 'serine/threonine kinase 35'}, {'end': 58130871L, 'name': 'AGAP2 p.A387T', 'start': 58130871L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'AGAP2', 'alt': 'T', 'ref': 'C', 'chromosome': '12', 'description': 'ArfGAP with GTPase domain, ankyrin repeat and PH domain 2'}, {'end': 33690771L, 'name': 'IP6K3 p.R320H', 'start': 33690771L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'IP6K3', 'alt': 'T', 'ref': 'C', 'chromosome': '6', 'description': 'inositol hexakisphosphate kinase 3'}, {'end': 101910837L, 'name': 'NALCN p.G408A', 'start': 101910837L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'NALCN', 'alt': 'G', 'ref': 'C', 'chromosome': '13', 'description': 'sodium leak channel, non-selective'}, {'end': 57424051L, 'name': 'MYO1A p.R845K', 'start': 57424051L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'MYO1A', 'alt': 'T', 'ref': 'C', 'chromosome': '12', 'description': 'myosin IA'}, {'end': 38211533L, 'name': 'TRPC4 p.R814K', 'start': 38211533L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'TRPC4', 'alt': 'T', 'ref': 'C', 'chromosome': '13', 'description': 'transient receptor potential cation channel, subfamily C, member 4'}, {'end': 32257834L, 'name': 'SPOCD1 p.R982C', 'start': 32257834L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'SPOCD1', 'alt': 'A', 'ref': 'G', 'chromosome': '1', 'description': 'SPOC domain containing 1'}, {'end': 28097282L, 'name': 'ZSCAN16 p.E201K', 'start': 28097282L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ZSCAN16', 'alt': 'A', 'ref': 'G', 'chromosome': '6', 'description': 'zinc finger and SCAN domain containing 16'}, {'end': 84628811L, 'name': 'SEMA3D p.R760Q', 'start': 84628811L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'SEMA3D', 'alt': 'T', 'ref': 'C', 'chromosome': '7', 'description': 'sema domain, immunoglobulin domain (Ig), short basic domain, secreted, (semaphorin) 3D'}, {'end': 72400932L, 'name': 'NEGR1 p.A80V', 'start': 72400932L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'NEGR1', 'alt': 'A', 'ref': 'G', 'chromosome': '1', 'description': 'neuronal growth regulator 1'}, {'end': 35778772L, 'name': 'ARPP21 p.P521L', 'start': 35778772L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ARPP21', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'cAMP-regulated phosphoprotein, 21kDa'}, {'end': 21016738L, 'name': 'KIF17 p.R442S', 'start': 21016738L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'KIF17', 'alt': 'T', 'ref': 'G', 'chromosome': '1', 'description': 'kinesin family member 17'}, {'end': 51929267L, 'name': 'IQCF1 p.R86Q', 'start': 51929267L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'IQCF1', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'IQ motif containing F1'}, {'end': 95188799L, 'name': 'CDH17 p.E132K', 'start': 95188799L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CDH17', 'alt': 'T', 'ref': 'C', 'chromosome': '8', 'description': 'cadherin 17, LI cadherin (liver-intestine)'}, {'end': 96117712L, 'name': 'CCDC82 p.L67P', 'start': 96117712L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CCDC82', 'alt': 'G', 'ref': 'A', 'chromosome': '11', 'description': 'coiled-coil domain containing 82'}, {'end': 105967614L, 'name': 'AASDHPPT p.R304*', 'start': 105967614L, 'biomarker_type': 'Nonsense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'AASDHPPT', 'alt': 'T', 'ref': 'C', 'chromosome': '11', 'description': 'aminoadipate-semialdehyde dehydrogenase-phosphopantetheinyl transferase'}, {'end': 31538698L, 'name': 'CLDN17 p.A80S', 'start': 31538698L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CLDN17', 'alt': 'A', 'ref': 'C', 'chromosome': '21', 'description': 'claudin 17'}, {'end': 38483424L, 'name': 'UTP11L p.M70I', 'start': 38483424L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'UTP11L', 'alt': 'A', 'ref': 'G', 'chromosome': '1', 'description': 'UTP11-like, U3 small nucleolar ribonucleoprotein (yeast)'}, {'end': 155526050L, 'name': 'FGG p.Q433P', 'start': 155526050L, 'biomarker_type': 'Splice_Site', 'referenceName': 'GRCh37', 'geneSymbol': 'FGG', 'alt': 'G', 'ref': 'T', 'chromosome': '4', 'description': 'fibrinogen gamma chain'}, {'end': 148617051L, 'name': 'ABLIM3 p.A310V', 'start': 148617051L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ABLIM3', 'alt': 'T', 'ref': 'C', 'chromosome': '5', 'description': 'actin binding LIM protein family, member 3'}, {'end': 31895941L, 'name': 'CFB p.P86S', 'start': 31895941L, 'biomarker_type': 'Splice_Site', 'referenceName': 'GRCh37', 'geneSymbol': 'CFB', 'alt': 'T', 'ref': 'C', 'chromosome': '6', 'description': 'complement factor B'}, {'end': 18051501L, 'name': 'MYO15A p.A2223V', 'start': 18051501L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'MYO15A', 'alt': 'T', 'ref': 'C', 'chromosome': '17', 'description': 'myosin XVA'}, {'end': 60969271L, 'name': 'CABLES2 p.S219C', 'start': 60969271L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CABLES2', 'alt': 'C', 'ref': 'G', 'chromosome': '20', 'description': 'Cdk5 and Abl enzyme substrate 2'}, {'end': 186096984L, 'name': 'KIAA1430 p.L426F', 'start': 186096984L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'KIAA1430', 'alt': 'A', 'ref': 'G', 'chromosome': '4', 'description': ''}, {'end': 110056501L, 'name': 'FIG4 p.G216R', 'start': 110056501L, 'biomarker_type': 'Splice_Site', 'referenceName': 'GRCh37', 'geneSymbol': 'FIG4', 'alt': 'A', 'ref': 'G', 'chromosome': '6', 'description': 'FIG4 phosphoinositide 5-phosphatase'}, {'end': 67690336L, 'name': 'RLTPR p.C1275Y', 'start': 67690336L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'RLTPR', 'alt': 'A', 'ref': 'G', 'chromosome': '16', 'description': 'RGD motif, leucine rich repeats, tropomodulin domain and proline-rich containing'}, {'end': 67702418L, 'name': 'C16orf86 p.D290G', 'start': 67702418L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'C16orf86', 'alt': 'G', 'ref': 'A', 'chromosome': '16', 'description': 'chromosome 16 open reading frame 86'}]

dataset_features = p204

dataset_name = 'p204'

In [78]:

def allele_identifier(feature):
    is_HG37 = re.compile('NC_.*\.10:g')    
    for s in feature.get('synonyms', []):
        if is_HG37.match(s):
            return '{} {}'.format(feature['geneSymbol'], s)
    return '{} {} {}'.format(feature['geneSymbol'], feature['start'], feature['sequence_ontology']['name'])


def to_df(generator, features ):
    
    def to_dict(generator, features):
        for hit in generator:
            drugs = [ec['term'].encode('utf-8') for ec in hit['association']['environmentalContexts']]
            description = hit['association']['description'].encode('utf-8')
            source = hit['source'].encode('utf-8')
            evidence_label = hit['association']['evidence_label']
            id = hit['evidence.id']
            phenotype = hit['association']['phenotype']['type']['term'].encode('utf-8')
            publications = []
            matches = []
            
            
            try:        
                for f in features:
                    matches.append(allele_identifier(f))
                    for f2 in hit['features']:
                        for s in f.get('synonyms', []):
                            if s in f2.get('synonyms', []):
                                matches.append(s)
                            for p in f.get('pathways',[]):
                                if p in f2.get('pathways',[]):
                                    # matches.append(p)
                                    matches.append('pathway')
                        for sp in f.get('swissprots',[]):
                            if sp in f2.get('swissprots',[]):
                                matches.append(sp)

                        if f.get('sequence_ontology', {'name':'X'}) == f2.get('sequence_ontology', {'name':None}):
                            matches.append(f['sequence_ontology']['name'])

                        if f['geneSymbol'] == f2.get('geneSymbol', None):
                            matches.append(f['geneSymbol'])
                            
                matches = list(set(matches))
                
                for e in hit['association']['evidence']:
                    for p in e['info']['publications']:
                        publications.append(p)


                yield {'id':id,
                       'source': source,
                       'evidence_label': evidence_label,
                       'drugs': drugs,
                       'description': description,
                       'phenotype':phenotype,
                       'publications':publications,
                       'matches': matches
                      }
            except Exception as e:
                print '!!!',e
                
    
    try:        
        return pd.DataFrame.from_records( to_dict(generator, features) , index='id')
    except Exception as e:
        return pd.DataFrame()
    

def allele_identifierOLD(feature_associations):
    is_HG37 = re.compile('NC_.*\.10:g')    
    for fa in feature_associations:
        for f in fa['features']:
            for s in f.get('synonyms', []):
                if is_HG37.match(s):
                    return '{} {}'.format(f['geneSymbol'], s)
            return '{} {} {}'.format(f['geneSymbol'], f['start'], f['sequence_ontology']['name'])

        
def biomarker_type(feature_associations):
    for fa in feature_associations:
        for f in fa['features']:
            return f['biomarker_type']
        

In [79]:

g2p = g2p_client.G2PDatabase('elastic')

queries = []
data_frames = {}

for patient_feature in dataset_features:
    location_query = g2p_client.location_query([patient_feature])
    identifier = allele_identifier(location_query['feature_associations'][0]['features'][0])
    b_type = biomarker_type(location_query['feature_associations'])
    for name in location_query['queries'].keys():
        qs = location_query['queries'][name]['query']['query_string']['query']
        if identifier not in data_frames:
            data_frames[identifier] = {}
        data_frames[identifier][name] = to_df(g2p.raw_dataframe(query_string=qs, verbose=False),
                                              location_query['feature_associations'][0]['features'])
        queries.append({'biomarker_type': b_type,
                        'allele': identifier,
                        'name': name,
                        'hits': len(data_frames[identifier][name]),
                        'query_string': qs
                       })



http://reg.genome.network/allele?hgvs=NC_000003.11%3Ag.41266125C%3ET
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000411226.1:p.Thr34Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000409302.1:p.Thr41Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000412219.1:p.Thr41Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000379486.3:p.Thr41Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000401599.1:p.Thr41Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000385604.1:p.Thr41Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000379488.3:p.Thr41Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000400508.1:p.Thr34Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000344456.5:p.Thr41Ile?domains

http://reg.genome.network/allele?hgvs=NC_000003.11%3Ag.142188239T%3EC
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000343741.4:p.Ile2164Met?domains=1&protein=1&uniprot=1
http://www.pathwaycommons.org/pc2/search.json?q=Q13535&organism=homo%20sapiens


http://reg.genome.network/allele?hgvs=NC_000010.10%3Ag.89685309_89685310insAATCT


http://reg.genome.network/allele?hgvs=NC_000003.11%3Ag.41266125C%3ET
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000411226.1:p.Thr34Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000409302.1:p.Thr41Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000412219.1:p.Thr41Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000379486.3:p.Thr41Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000401599.1:p.Thr41Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000385604.1:p.Thr41Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000379488.3:p.Thr41Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000400508.1:p.Thr34Ile?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000344456.5:p.Thr41Ile?domains

http://reg.genome.network/allele?hgvs=NC_000023.10%3Ag.1584977C%3ET
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000370697.4:p.Ala159Thr?domains=1&protein=1&uniprot=1
http://www.pathwaycommons.org/pc2/search.json?q=Q86VZ1&organism=homo%20sapiens


...

http://reg.genome.network/allele?hgvs=NC_000006.11%3Ag.31895941C%3ET
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000410815.1:p.Pro86Ser?domains=1&protein=1&uniprot=1
{u'error': u"Unable to parse HGVS notation 'ENSP00000410815.1:p.Pro86Ser':  : Unable to map the peptide coordinate 86 to genomic coordinates for protein ENSP00000410815: Could not get a Transcript object for 'ENSP00000410815'"} https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000410815.1:p.Pro86Ser?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000403325.1:p.Pro86Ser?domains=1&protein=1&uniprot=1
{u'error': u"Unable to parse HGVS notation 'ENSP00000403325.1:p.Pro86Ser':  : Unable to map the peptide coordinate 86 to genomic coordinates for protein ENSP00000403325: Could not get a Transcript object for 'ENSP00000403325'"} https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000403325.1:p.Pro86Ser?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP

http://reg.genome.network/allele?hgvs=NC_000017.10%3Ag.18051501C%3ET
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000481642.1:p.Ala2223Val?domains=1&protein=1&uniprot=1
{u'error': u"Unable to parse HGVS notation 'ENSP00000481642.1:p.Ala2223Val': Could not get a Transcript object for 'ENSP00000481642'"} https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000481642.1:p.Ala2223Val?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000205890.5:p.Ala2223Val?domains=1&protein=1&uniprot=1
http://www.pathwaycommons.org/pc2/search.json?q=Q9UKN7&organism=homo%20sapiens
http://reg.genome.network/allele?hgvs=NC_000020.10%3Ag.60969271G%3EC
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000279101.5:p.Ser219Cys?domains=1&protein=1&uniprot=1
http://www.pathwaycommons.org/pc2/search.json?q=Q9BTV7&organism=homo%20sapiens


http://reg.genome.network/allele?hgvs=NC_000004.11%3Ag.186096984G%3EA
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000423312.1:p.Leu426Phe?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000409964.2:p.Leu426Phe?domains=1&protein=1&uniprot=1
http://www.pathwaycommons.org/pc2/search.json?q=Q9P2B7&organism=homo%20sapiens
http://reg.genome.network/allele?hgvs=NC_000006.11%3Ag.110056501G%3EA
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000230124.3:p.Gly216Arg?domains=1&protein=1&uniprot=1


{u'error': u"Unable to parse HGVS notation 'ENSP00000230124.3:p.Gly216Arg':  : Unable to map the peptide coordinate 216 to genomic coordinates for protein ENSP00000230124: Could not get a Transcript object for 'ENSP00000230124'"} https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000230124.3:p.Gly216Arg?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000412156.1:p.Gly195Arg?domains=1&protein=1&uniprot=1
{u'error': u"Unable to parse HGVS notation 'ENSP00000412156.1:p.Gly195Arg':  : Unable to map the peptide coordinate 195 to genomic coordinates for protein ENSP00000412156: Could not get a Transcript object for 'ENSP00000412156'"} https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000412156.1:p.Gly195Arg?domains=1&protein=1&uniprot=1
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000357937.1:p.Gly139Arg?domains=1&protein=1&uniprot=1
{u'error': u"Unable to parse HGVS notation 'ENSP00000357937.1:p.Gly139Arg':  : Unable to map the peptide coordin

http://reg.genome.network/allele?hgvs=NC_000016.9%3Ag.67702418A%3EG
https://grch37.rest.ensembl.org/vep/human/hgvs/ENSP00000384117.3:p.Asp290Gly?domains=1&protein=1&uniprot=1
http://www.pathwaycommons.org/pc2/search.json?q=Q9H0I2&organism=homo%20sapiens
http://www.pathwaycommons.org/pc2/search.json?q=Q6ZW13&organism=homo%20sapiens


In [80]:
from IPython.display import clear_output
from IPython.display import IFrame  
from IPython.core.display import HTML

queries_df = pd.DataFrame.from_records(queries)
queries_df_widget = qgrid.show_grid(queries_df, grid_options={'editable':False})
qgrid_widget = None
top3_widget = None
allele = None 
name = None

def top(df, fields = ['publications', 'evidence_label', 'phenotype', 'drugs' ], size=3):
    counters = []
    for f in fields:
        counter = Counter()
        for value in df[f]:
            if isinstance(value, (list,tuple,)):
                for v in value:
                    counter[v] += 1
            else:
                counter[value] += 1
        for c in counter.most_common(size):
            counters.append({'field': f, 'value': c[0], 'count': c[1]})
    return counters    
    

def on_queries_df_row_selected(change):
    global allele
    global name
    global qgrid_widget
    global top3_widget    
    selected_row = queries_df_widget.get_selected_df().to_dict('records')
    allele = selected_row[0]['allele']
    name = selected_row[0]['name']
    query_string = selected_row[0]['query_string']
    top3_widget = None
    
    try:    
        df = data_frames[allele][name]
        qgrid_widget = qgrid.show_grid(df)
        top3_widget = qgrid.show_grid(pd.DataFrame(top(df)))
        
    except Exception as e:
        # df could be empty
        pass  
    
    # show publication
    def on_row_selected(change):
        global qgrid_widget
        try:
            selected_row = qgrid_widget.get_selected_df().to_dict('records')
            publications = selected_row[0]['publications']
            matches = selected_row[0]['matches']            
            clear_output(wait=True)
            display(queries_df_widget,
                    HTML('<p>{} {}</p><p>{}</p>'.format(allele, name, query_string)),
                    qgrid_widget,
                    HTML('<p>matches:{}</p><p>{}</p>'.format(matches, publications)),
                    IFrame(publications[0], width=900, height=900))    
        except Exception as e:          
            print e

    qgrid_widget.observe(on_row_selected, names=['_selected_rows'])        
    clear_output(wait=True)
    try:
        if top3_widget:
            display(queries_df_widget,
                    HTML('<p>top3:</p>'),
                    top3_widget,
                    HTML('<p>{} {}</p><p>{}</p>'.format(allele, name, query_string)),
                    qgrid_widget)    
        else:
            display(queries_df_widget, HTML('<p>No results</p>'))
    except Exception as e:          
        print e        
    
queries_df_widget.observe(on_queries_df_row_selected, names=['_selected_rows'])   # ,  

display(queries_df_widget)

QgridWidget(grid_options={'defaultColumnWidth': 150, 'highlightSelectedRow': True, 'enableTextSelectionOnCells': True, 'editable': False, 'explicitInitialization': True, 'minVisibleRows': 8, 'enableColumnReorder': False, 'maxVisibleRows': 15, 'sortable': True, 'fullWidthRows': True, 'rowHeight': 28, 'filterable': True, 'autoEdit': False, 'syncColumnCellResize': True, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)

In [81]:

def collect_md(s, md=None):
    s = s.decode('UTF-8', 'ignore')
    if md:
        md = u'{}\n{}'.format(md, s)
    else:
        md = u'{}'.format(s)
    return md

# report parameters
query_names = ['alleles', '~location', '~range', 'protein_effects', 'protein_domain', '~biomarker_type',  'genes' , 'pathways']
evidence_levels = ['A', 'B', 'C', 'D']
# maximum number of matching evidence items
limit = 2

In [83]:

 
md = collect_md('# {}'.format(dataset_name))

md = collect_md('\n## Drug Centric\n_____', md)

for name in query_names:
    df = pd.DataFrame()
    for allele in data_frames.keys():
        if name in data_frames[allele] and len(data_frames[allele][name]):
            df = df.append(data_frames[allele][name])
    if len(df) == 0:
        continue
    md = collect_md('\n### {}\n'.format(name), md)
    for evidence_level in evidence_levels:
        evidence_level_df = df[df.evidence_label == evidence_level]
        if len(evidence_level_df):
            drug = top(evidence_level_df, fields=['drugs'], size=1)[0]['value']
            md = collect_md('  * {} {}'.format(evidence_level, drug), md)
            c = 0
            for evidence in evidence_level_df.to_records():
                if drug in evidence.drugs:
                    md = collect_md('    * {} {} {}'.format(evidence.source, evidence.phenotype, evidence.drugs), md)
                    md = collect_md('      * {}'.format(evidence.description), md)
                    md = collect_md('      * {}'.format(evidence.matches), md)                    
                    md = collect_md('      * {}'.format(evidence.publications), md)                                        
                    c += 1
                    if c == limit:
                        break
                        
md = collect_md('\n## Phenotype Centric\n_____', md)

for name in query_names:
    df = pd.DataFrame()
    for allele in data_frames.keys():
        if name in data_frames[allele] and len(data_frames[allele][name]):
            df = df.append(data_frames[allele][name])
    if len(df) == 0:
        continue
    md = collect_md('\n### {}\n'.format(name), md)
    for evidence_level in evidence_levels:
        evidence_level_df = df[df.evidence_label == evidence_level]
        if len(evidence_level_df):
            phenotype = top(evidence_level_df, fields=['phenotype'], size=1)[0]['value']
            md = collect_md('  * {} {}'.format(evidence_level, phenotype), md)
            c = 0
            for evidence in evidence_level_df.to_records():
                if phenotype in evidence.phenotype:
                    md = collect_md('    * {} {} {}'.format(evidence.source, evidence.phenotype, evidence.drugs), md)
                    md = collect_md('      * {}'.format(evidence.description), md)
                    md = collect_md('      * {}'.format(evidence.matches), md)                    
                    md = collect_md('      * {}'.format(evidence.publications), md)                                        
                    c += 1
                    if c == limit:
                        break

                        
                        
md = collect_md('\n## Publication Centric\n_____', md)

for name in query_names:
    df = pd.DataFrame()
    for allele in data_frames.keys():
        if name in data_frames[allele] and len(data_frames[allele][name]):
            df = df.append(data_frames[allele][name])
    if len(df) == 0:
        continue
    md = collect_md('\n### {}\n'.format(name), md)
    for evidence_level in evidence_levels:
        evidence_level_df = df[df.evidence_label == evidence_level]
        if len(evidence_level_df):
            publication = top(evidence_level_df, fields=['publications'], size=1)[0]['value']
            md = collect_md('  * {} {}'.format(evidence_level, publication), md)
            c = 0
            for evidence in evidence_level_df.to_records():
                if publication in evidence.publications:
                    md = collect_md('    * {} {} {}'.format(evidence.source, evidence.phenotype, evidence.drugs), md)
                    md = collect_md('      * {}'.format(evidence.description), md)
                    md = collect_md('      * {}'.format(evidence.matches), md)                   
                    md = collect_md('      * {}'.format(evidence.publications), md)                    
                    c += 1
                    if c == limit:
                        break

                        
                        
# print md    
Markdown(md)

